# Reconstruct 3D image of suspended cells for Figure 4b
Data: 3D bluechannel image of suspended MDCK

## Libraries

In [1]:
import numpy as np
import myfunctions as myfunc
from astropy.io import fits
from skimage.transform import rescale
import matplotlib.pyplot as plt

## Functions

In [ ]:
def reconstruction(data, suffix, raw, calib, result, wavelength,
                   ex, bg, theta, pixelsize, mag, fps, v, F_indexmismatch, polarity, coeff_405,
                   ex_bg_488=file_488_ex_bg, print_loop=False, kernel_movmean=10, shift1=(0, 0, 0), rescaling_factor=0.25):
    '''
    Reconstruct 3D image

    data: data name of the input data
    suffix: suffix of the data
    raw: path of raw data
    calib: path of calibration data
    result: path of result
    wavelength: wavelength of excitation (405, 488)
    ex: excitation profile
    bg: background profile
    theta: angle of light sheet in water
    pixelsize: pixel size of the image sensor in um
    mag: magnification rate of imaging
    fps: frame per second in Hz
    v: scanning velocity in the flow direction in um/s
    F_indexmismatch: axial elongation factor due to refractive-index mismatch of remote focusing
    polarity: forward or reverse scanning direction
    coeff_405: coefficient to calibrate aberration of blue channel image
    ex_bg_488: background profile of 488 excitation
    print_loop: whether to print loop counter
    kernel_movmean: bin width to calculate background
    shift1: shit of 405 image for spatial calibration
    rescaling_factor: factor to rescale image
    '''
    if wavelength=='405':
        for s in suffix:
            print(s)
            file = path_rawdata+data_name+s+'.fits'
            img = fits.getdata(file)
            (dim_flow, dim_height, dim_width) = img.shape

            # prepare an array to store the reconstructed image
            T, dim_height_recon, dim_flow_recon = myfunc.get_affine_parameters(theta,pixelsize,mag, fps, v, F_indexmismatch/coeff_405, polarity, dim_height, dim_flow)
            img_stack = np.float32(np.transpose(img, [1,2,0]))
            img_out = np.float32(np.zeros((dim_height_recon, dim_width, dim_flow_recon)))
            print('initialized')

            # calculate background and excitation profile
            img_bg, img_ex =  myfunc.calc_bg_ex_profiles(calib+bg, calib+ex, calib+ex_bg_488, kernel_movmean)
            
            if polarity == 1:
                img_stack = np.flip(img_stack, 0)

            # calibrate the input image
            myfunc.bg_ex_calib(img_stack, img_bg, img_ex, dim_flow, '405', print_loop)

            # reconstruct
            myfunc.affine_transform_3D(img_out, img_stack[:,:,0:dim_flow-1], T[:2], dim_width, dim_flow_recon, dim_height_recon, '405', print_loop)

            # rescale the image
            if rescaling_factor!=1:
                img_out = rescale(img_out, rescaling_factor, anti_aliasing=True)

            # spatially calibrate the image
            img_out = np.roll(img_out, shift1, axis=(0,1,2))       
            img_out = np.flip(img_out, axis=0)

            np.save(f'{result}{data}{s}.npy', img_out)

    if wavelength=='488':
        for s in suffix:
            print(s)
            file = path_rawdata+data_name+s+'.fits'
            img = fits.getdata(file)
            (dim_flow, dim_height, dim_width) = img.shape
            
            T, dim_height_recon, dim_flow_recon = myfunc.get_affine_parameters(theta,pixelsize,mag, fps, v, F_indexmismatch, polarity, dim_height, dim_flow)
            img_stack = np.float32(np.transpose(img, [1,2,0]))
            img_out = np.float32(np.zeros((dim_height_recon, dim_width, dim_flow_recon)))
            print('initialized')

            img_bg, img_ex =  myfunc.calc_bg_ex_profiles(calib+bg, calib+ex, calib+ex_bg_488, kernel_movmean)
            
            if polarity == 1:
                img_stack = np.flip(img_stack, 0)
            
            myfunc.bg_ex_calib(img_stack, img_bg, img_ex, dim_flow, '488', print_loop)
            
            myfunc.affine_transform_3D(img_out, img_stack[:,:,0:dim_flow-1], T[:2], dim_width, dim_flow_recon, dim_height_recon, '488', print_loop)
            
            if rescaling_factor!=1:
                img_out = rescale(img_out, rescaling_factor, anti_aliasing=True)
            
            img_out = np.flip(img_out, axis=0)

            np.save(f'{result}{data}{s}.npy', img_out)

## Parameters

In [ ]:
# acquisition parameters
theta = np.arcsin(0.62/1.33)
pixelsize = 6.5
mag = 200/9
fps = 300
F_indexmismatch = 1.412
polarity =  0
coeff_405 = 0.994

In [1]:
# file names for calibrartion
file_405_ex = 'ex_405.tif'
file_488_ex = 'ex_488.tif'
file_488_ex_bg = 'bg_1Hz_100average_488.tif'
file_405_bg = 'bg_300Hz_100average_405.tif'
file_488_bg = 'bg_300Hz_100average_488.tif'

In [3]:
suffix = ['', '_X1', '_X2', '_X3', '_X4', '_X5']

## Directories

In [2]:
calib = '../data/3D/calib/'
raw = '../data/3D/raw/'
result = '../result/'

## Analysis

### Measurement 1

In [ ]:
file_405 = 'id551_MDCK_cellonly_01_300Hz_405_250mW_488_150mW_405'
file_488 = 'id551_MDCK_cellonly_01_300Hz_405_250mW_488_150mW_488'

In [4]:
# velocity calculation
mean_aspect_ratio = 7.84 # mean aspect ratio of beads
v = mean_aspect_ratio*fps*(pixelsize/mag)

NameError: name 'fps' is not defined

In [8]:
reconstruction(data_name=file_405, suffix=suffix, wavelength='405', ex=file_405_ex, bg=file_405_bg, theta=theta,
               pixelsize=pixelsize, mag=mag, fps=fps, v=v, F_indexmismatch=F_indexmismatch, polarity=polarity, coeff_405=coeff_405,
               ex_bg_488=file_488_ex_bg, print_loop=False, kernel_movmean=10, shift1=(0, 0, 0), rescaling_factor=0.25)


initialized
_X1
initialized
_X2
initialized
_X3
initialized
_X4
initialized
_X5
initialized
_X6
initialized
